In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Longevity_and_Aging/GSE16717'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles in the Leiden Longevity Study"
!Series_summary	"Biomarkers of familial longevity may represent mechanisms underlying healthy aging. To identify gene expression profiles marking human familial longevity, an explorative genome-wide expression study was performed among 50 families from the Leiden Longevity Study who have a life-long survival advantage of 30%.  Gene expression profiles were compared between 50 nonagenarians (mean age 93.4 years) and 50 controls (mean age 61.9 years) to investigate differential gene expression that may arise as a function of both chronological age and familial longevity. Differential expression was observed for 2953 probes (FDR≤0.05) and for 109 GO terms, which corresponded well with previously reported findings on gene expression changes associated with chronological age, such as ‘immune response’, ‘signal transduction’ and ‘regulation of gene transcription’. To explore which of the 2953 chron

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # based on the series summary description of gene expression profiles

# Determine the availability and keys for required variables
sample_characteristics = {0: ['group: long-lived sib', 'group: control', 'group: offspring'], 
                          1: ['gender: female', 'gender: male'], 
                          2: ['age: 91.53 years', 'age: 56.1 years', 'age: 91.52 years', 'age: 52.83 years', 
                              'age: 64.11 years', 'age: 64.27 years', 'age: 59.75 years', 'age: 93.4 years', 
                              'age: 61.47 years', 'age: 93.19 years', 'age: 90.79 years', 'age: 53.4 years', 
                              'age: 96.75 years', 'age: 101.16 years', 'age: 98.26 years', 'age: 54.37 years', 
                              'age: 58.01 years', 'age: 59.93 years', 'age: 60.73 years', 'age: 92.76 years', 
                              'age: 62.88 years', 'age: 69.31 years', 'age: 90.22 years', 'age: 89.52 years', 
                              'age: 63.1 years', 'age: 56.93 years', 'age: 91.74 years', 'age: 90.37 years', 
                              'age: 94.33 years', 'age: 60.31 years'], 
                          3: ['family: Family 118', 'family: Family 142', 'family: Family 129', 'family: Family 008', 
                              'family: Family 136', 'family: Family 181', 'family: Family 085', 'family: Family 108', 
                              'family: Family 188', 'family: Family 056', 'family: Family 025', 'family: Family 123', 
                              'family: Family 380', 'family: Family 189', 'family: Family 228', 'family: Family 196', 
                              'family: Family 171', 'family: Family 066', 'family: Family 257', 'family: Family 305', 
                              'family: Family 063', 'family: Family 014', 'family: Family 371', 'family: Family 156', 
                              'family: Family 049', 'family: Family 264', 'family: Family 420', 'family: Family 016', 
                              'family: Family 295', 'family: Family 291'], 
                          4: ['hybridization day: Day 01', 'hybridization day: Day 02', 'hybridization day: Day 03', 
                              'hybridization day: Day 06', 'hybridization day: Day 04', 'hybridization day: Day 05', 
                              'hybridization day: Day 07', 'hybridization day: Day 08', 'hybridization day: Day 09', 
                              'hybridization day: Day 10', 'hybridization day: Day 11', 'hybridization day: Day 12', 
                              'hybridization day: Day 13', 'hybridization day: Day 14']}

# Longevity trait information
trait_row = 0  # as inferred from 'group: long-lived sib', 'group: control', 'group: offspring'

# Age information
age_row = 2  # as inferred from the age-related values listed

# Gender information
gender_row = 1  # as inferred from 'gender: female', 'gender: male'

# Functions for data type conversion
def convert_trait(value):
    if 'group: long-lived sib' in value:
        return 1
    elif 'group: control' in value or 'group: offspring' in value:
        return 0
    return None

def convert_age(value):
    try:
        return float(value.split(': ')[1].split()[0])
    except ValueError:
        return None

def convert_gender(value):
    if 'gender: female' in value:
        return 0
    elif 'gender: male' in value:
        return 1
    return None

# Save cohort information
save_cohort_info('GSE16717', './preprocessed/Longevity_and_Aging/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Longevity_and_Aging', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Longevity_and_Aging/trait_data/GSE16717.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM418770': [1.0, 91.53, 0.0], 'GSM418771': [0.0, 56.1, 0.0], 'GSM418772': [1.0, 91.52, 1.0], 'GSM418773': [0.0, 52.83, 1.0], 'GSM418774': [0.0, 64.11, 0.0], 'GSM418775': [0.0, 64.27, 1.0], 'GSM418776': [0.0, 59.75, 1.0], 'GSM418777': [1.0, 93.4, 0.0], 'GSM418778': [0.0, 61.47, 1.0], 'GSM418779': [1.0, 93.19, 1.0], 'GSM418780': [1.0, 90.79, 1.0], 'GSM418781': [0.0, 53.4, 0.0], 'GSM418782': [1.0, 96.75, 0.0], 'GSM418783': [1.0, 101.16, 0.0], 'GSM418784': [1.0, 98.26, 0.0], 'GSM418785': [0.0, 54.37, 0.0], 'GSM418786': [0.0, 58.01, 0.0], 'GSM418787': [0.0, 59.93, 0.0], 'GSM418788': [0.0, 60.73, 1.0], 'GSM418789': [1.0, 92.76, 0.0], 'GSM418790': [0.0, 62.88, 0.0], 'GSM418791': [0.0, 69.31, 1.0], 'GSM418792': [1.0, 90.22, 1.0], 'GSM418793': [1.0, 89.52, 1.0], 'GSM418794': [0.0, 63.1, 1.0], 'GSM418795': [0.0, 56.93, 0.0], 'GSM418796': [1.0, 91.74, 1.0], 'GSM418797': [1.0, 90.37, 1.0], 'GSM418798': [1.0, 94.33, 0.0], 'GSM418799': [0.0, 60.31, 0.0], 'GSM418800': [0.0, 64.62, 1.0], 'GSM418801

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1005', '1006', '1007', '1010', '1011', '1014', '1023', '1027', '1028',
       '1048', '1054', '1059', '1061', '1065', '1072', '1073', '1074', '1076',
       '1079', '1080'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1001', '1002', '1003', '1004', '1005'], 'LOGICAL_ROW': ['1', '1', '1', '1', '1'], 'LOGICAL_COL': [1.0, 2.0, 3.0, 4.0, 5.0], 'PROBE_NAME': ['GE200017', 'GE766244', 'GE766859', 'GE519376', 'GE519777'], 'PROBE_TYPE': ['FIDUCIAL', 'DISCOVERY', 'DISCOVERY', 'DISCOVERY', 'DISCOVERY'], 'PUB_PROBE_TARGETS': ['CONTROL', 'SINGLE', 'SINGLE', 'SINGLE', 'SINGLE'], 'SPOT_ID': ['-- FID CTRL: PCTRL17', nan, nan, '-- NP DISC: INCYTE UNIQUE', nan], 'GB_LIST': [nan, 'XM_293099.2', 'BF588963.1', nan, 'BE550764.1'], 'GI_LIST': [nan, '30157495', '11681287', nan, '9792456']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. When analyzing a gene expression dataset, we need to map some identifiers of genes to actual gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'GB_LIST'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Longevity_and_Aging/gene_data/GSE16717.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Longevity_and_Aging')

# 4. Save the cohort information.
save_cohort_info('GSE16717', './preprocessed/Longevity_and_Aging/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Longevity_and_Aging/GSE16717.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
